In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Proposed Algo

In [35]:
"""Contains custom exceptions for LSB Steganography program"""


class FileError(Exception):
    pass

class DataError(Exception):
    pass

class SkinNotDetected(Exception):
    pass

class LargestComponentNotFound(Exception):
    pass

class SeedNotValid(Exception):
    pass

class NotEnoughCapasity(Exception):
    pass


In [36]:
# -*- coding: utf-8 -*-
"""SDP_FINAL124.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ydDP8-xTYAr2I1g-bdifQwYZG9eR5m3K

# Function Defination
"""

# Libraries
import datetime
import pathlib
import time
import cv2
import numpy as np
import pywt
import skimage.measure
import math
import random
from random import randint, seed
from sewar.full_ref import ssim, uqi, psnr

# region
"""##Skin Detection"""


def skin_detection(img):
    # converting from gbr to hsv color space
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # skin color range for hsv color space
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17, 170, 255))
    HSV_mask = cv2.morphologyEx(
        HSV_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    # skin color range for YCbCr color space
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255, 180, 135))
    YCrCb_mask = cv2.morphologyEx(
        YCrCb_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # merge skin detection (YCbCr and hsv)
    global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
    # removing noise from image
    global_mask = cv2.medianBlur(global_mask, 3)
    global_mask = cv2.morphologyEx(
        global_mask, cv2.MORPH_OPEN, np.ones((4, 4), np.uint8))
    # checking if there are skin pixels in image or not
    unique, counts = np.unique(global_mask, return_counts=True)
    if(unique[-1] != 255):
        raise SkinNotDetected(
            "Oops!!! Skin can not be Detected in given Image")
    # cv2.imwrite("Skin_Detection.jpeg", global_mask)
    return global_mask


"""##Hole Filling"""


def FillHole(mask):

    contours, hierarchy = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    len_contour = len(contours)
    contour_list = []
    for i in range(len_contour):
        drawing = np.zeros_like(mask, np.uint8)  # create a black image
        img_contour = cv2.drawContours(
            drawing, contours, i, (255, 255, 255), -1)
        contour_list.append(img_contour)
    out = sum(contour_list)
    if type(out) == type(1):
        if(out == 0):
            raise SkinNotDetected(
                "Oops!!! Hole Filling is not Possible in this Image")
    # cv2.imwrite("Hole_Filling.jpg", out)

    img = out
    img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
    num_labels, labels_im = cv2.connectedComponents(img)

    # Map component labels to hue val
    label_hue = np.uint8(179*labels_im/np.max(labels_im))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue == 0] = 0

    # cv2.imwrite("Connected_Component.jpg", labeled_img)

    return out


"""##Largest Connected Component"""


def largest_connected_component_coordinates(image):
    image = image.astype("uint8")
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(
        image, connectivity=8
    )
    # states has sizes of all connected componets there is in the image and we will take it as non-increasing array
    sizes = stats[:, -1]
    # print(sizes)
    # find component of size >3000
    """
    print(sizes)
    if(sizes[1] <= 3000):
        # if no component is found then we can not proceed further with the process
        sys.exit("There is no connected component of size greater than 3000")
    else:
        max_label = 1
        max_size = sizes[1]
    """

    max_label = 1
    max_size = 3000  # find component of size >3000
    # 0th component is the background component so we cannot take that
    for i in range(1, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    if(max_size == 3000):
        raise LargestComponentNotFound(
            "Oops!!! Skin Component larger than 3000 can't be Found")
    # cv2.imwrite("Largest_Connected_Component.jpg", output)
    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
    y_nonzero, x_nonzero = np.nonzero(img2)
    return (y_nonzero, x_nonzero)


"""##Crop ROI"""


def crop(y_nonzero, x_nonzero, image):
    # cv2.imwrite("Crop.jpg", image[np.min(y_nonzero): np.max(
    #     y_nonzero), np.min(x_nonzero): np.max(x_nonzero)])

    print(np.min(y_nonzero), np.max(y_nonzero),
          np.min(x_nonzero), np.max(x_nonzero))
    return image[
        np.min(y_nonzero): np.max(y_nonzero), np.min(x_nonzero): np.max(x_nonzero)
    ]


"""##Get sub-bands"""


def get_subbands(ROI):
    coeffs = pywt.dwt2(ROI, "haar")
    cA, (cH, cV, cD) = coeffs
    # cv2.imwrite("cA.jpg", cA)
    # cv2.imwrite("cV.jpg", cV)
    # cv2.imwrite("cH.jpg", cH)
    # cv2.imwrite("cD.jpg", cD)
    return coeffs


"""##Get maxEntropy sub-band"""


def get_max_entropy_subband(coeffs):
    cA, (cH, cV, cD) = coeffs
    x, y = cH.shape
    print("Shape of Subnand :", x, " x ", y)
    Dict = {
        "cH": skimage.measure.shannon_entropy(cH),
        "cV": skimage.measure.shannon_entropy(cV),
        "cD": skimage.measure.shannon_entropy(cD),
    }
    print("LH : ", skimage.measure.shannon_entropy(cH))
    print("HL : ", skimage.measure.shannon_entropy(cV))
    print("HH : ", skimage.measure.shannon_entropy(cD))
    # find Maximum from Dictionary
    Keymax = max(zip(Dict.values(), Dict.keys()))[1]
    if Keymax == "cH":
        print("Selected Band : LH")
        # cv2.imwrite("Max_Entropy_cH.jpg", cH)
        maxEntropy_Band = cH
    elif Keymax == "cV":
        print("Selected Band : HL")
        # cv2.imwrite("Max_Entropy_cV.jpg", cV)
        maxEntropy_Band = cV
    elif Keymax == "cD":
        print("Selected Band : HH")
        # cv2.imwrite("Max_Entropy_cD.jpg", cD)
        maxEntropy_Band = cD

    return maxEntropy_Band


"""##Generate binary steam from string"""


def str_to_binary(string):
    string = str(string)
    binary = "".join(format(i, "08b")
                     for i in bytearray(string, encoding="utf-8"))
    return binary


"""##Generate Coordinates"""


def generate_coordinates(ROI, length):
    m, n = ROI.shape
    m = m - 1
    n = n - 1
    random.seed(length)
    coordinates = set()
    x, y = randint(1, m), randint(1, n)
    while len(coordinates) < length:
        while (x, y) in coordinates:
            x, y = randint(1, m), randint(1, n)
        coordinates.add((x, y))
    return coordinates


"""##Hiding inside Max Entropy Band"""


def hide_in_maxband(coordinates, maxEntropy_Band, binary):
    modified_maxEntropy_Band = np.copy(maxEntropy_Band)
    i = 0
    for x, y in coordinates:

        fract_pixel, int_pixel = math.modf(maxEntropy_Band[x, y])
        bin_pixel = bin(int(int_pixel))
        new_bin_pixel = list(bin_pixel)
        new_bin_pixel[len(bin_pixel) - 1] = binary[i]
        bin_pixel = "".join(new_bin_pixel)
        int_pixel = int(bin_pixel, 2)
        modified_maxEntropy_Band[x, y] = int_pixel + fract_pixel
        i = i + 1
    # cv2.imwrite("Modified_Subband.jpg", modified_maxEntropy_Band)
    return modified_maxEntropy_Band


"""##Get ROI back by Applying IDWT"""


def get_modified_ROI(coeffs, modified_max_entropy_band, maxEntropy_Band):
    cA, (cH, cV, cD) = coeffs
    comparison = maxEntropy_Band == cH
    equal_arrays_cH = comparison.all()
    comparison = maxEntropy_Band == cV
    equal_arrays_cV = comparison.all()
    comparison = maxEntropy_Band == cD
    equal_arrays_cD = comparison.all()

    if equal_arrays_cH:
        modified_cH = modified_max_entropy_band
        modified_coeffs = cA, (modified_cH, cV, cD)
    elif equal_arrays_cV:
        modified_cV = modified_max_entropy_band
        modified_coeffs = cA, (cH, modified_cV, cD)
    elif equal_arrays_cD:
        modified_cD = modified_max_entropy_band
        modified_coeffs = cA, (cH, cV, modified_cD)

    # Applying Invers IDWT (Embedding Side)
    idwt_ROI = pywt.idwt2(modified_coeffs, "haar")
    # cv2.imwrite("idwt_ROI.jpg", idwt_ROI)
    return idwt_ROI


"""##Seperating Intger and Fractional part of the modified ROI"""


def seperate_int_fract(idwt_ROI):
    rows, cols = idwt_ROI.shape
    int_ROI = np.empty((rows, cols), np.uint8)
    fract_ROI = np.empty((rows, cols), np.float64)

    for x in range(0, rows):
        for y in range(0, cols):
            fract_pixel, int_pixel = math.modf(idwt_ROI[x, y])
            if int_pixel >= 0:
                int_ROI[x, y] = int_pixel
                fract_ROI[x, y] = fract_pixel
            else:
                int_ROI[x, y] = abs(int_pixel)
                fract_ROI[x, y] = -fract_pixel
    #cv2.imwrite("Int_Part.png", int_ROI)
    # cv2.imwrite("Fract_Part.jpg", fract_ROI)
    return int_ROI, fract_ROI


"""##Merging modified ROI back to binary image"""


def merge_roi(binary_image, int_ROI, y_nonzero, x_nonzero):
    x = np.min(x_nonzero)
    y = np.min(y_nonzero)
    binary_image[y: y + int_ROI.shape[0], x: x + int_ROI.shape[1]] = int_ROI
    # cv2.imwrite("Merged_Image.jpg", binary_image)
    return binary_image


"""##Encoding function for hiding binary image into R-plane"""


def hide_in_R(image_to_hide, image_to_hide_in):
    width, height = image_to_hide_in.shape

    for y in range(height):
        for x in range(width):
            value = image_to_hide[x, y]
            pixel = image_to_hide_in[x, y]
            bin_pixel = bin(int(pixel))
            new_bin_pixel = list(bin_pixel)
            if value == 0:
                new_bin_pixel[len(bin_pixel) - 2] = "0"
                new_bin_pixel[len(bin_pixel) - 1] = "0"
            elif value == 1:
                new_bin_pixel[len(bin_pixel) - 2] = "0"
                new_bin_pixel[len(bin_pixel) - 1] = "1"
            elif value == 254:
                new_bin_pixel[len(bin_pixel) - 2] = "1"
                new_bin_pixel[len(bin_pixel) - 1] = "0"
            elif value == 255:
                new_bin_pixel[len(bin_pixel) - 2] = "1"
                new_bin_pixel[len(bin_pixel) - 1] = "1"

            bin_pixel = "".join(new_bin_pixel)
            int_pixel = int(bin_pixel, 2)
            image_to_hide_in[x, y] = int_pixel

    # return an Image object from the above data.
    # cv2.imwrite("Hide_in_R_plane.jpg", image_to_hide_in)
    return image_to_hide_in


# This code retrives image from an image using 2-LSB


def get_binary_from_R(combine_img):

    width, height = combine_img.shape
    hidden_img = np.zeros_like(combine_img, dtype="uint8")
    for y in range(height):
        for x in range(width):
            pixel = combine_img[x, y]
            bin_pixel = bin(int(pixel))
            new_bin_pixel = list(bin_pixel)
            if (
                new_bin_pixel[len(bin_pixel) - 2] == "0"
                and new_bin_pixel[len(bin_pixel) - 1] == "0"
            ):
                value = 0
            elif (
                new_bin_pixel[len(bin_pixel) - 2] == "0"
                and new_bin_pixel[len(bin_pixel) - 1] == "1"
            ):
                value = 1
            elif (
                new_bin_pixel[len(bin_pixel) - 2] == "1"
                and new_bin_pixel[len(bin_pixel) - 1] == "0"
            ):
                value = 254
            elif (
                new_bin_pixel[len(bin_pixel) - 2] == "1"
                and new_bin_pixel[len(bin_pixel) - 1] == "1"
            ):
                value = 255

            hidden_img[x, y] = value
    # return an Image object from the above data.
    return hidden_img


"""##combine fact and int ROI"""


def combine_roi(ROI, fract):
    width, height = ROI.shape
    new_roi = np.zeros_like(ROI, dtype="float64")

    for y in range(height):
        for x in range(width):
            fractd = fract[x, y]
            if fractd < 0:
                new_roi[x, y] = -(ROI[x, y] + abs(fractd))
            else:
                new_roi[x, y] = ROI[x, y] + fractd
    return new_roi


"""##Get Message"""


def get_message(maxEntropy_Band, coordinates):
    message = []
    for x, y in coordinates:
        pixel = maxEntropy_Band[x, y]
        fract_pixel, int_pixel = math.modf(maxEntropy_Band[x, y])
        bin_pixel = bin(int(int_pixel))
        message.append(bin_pixel[len(bin_pixel) - 1])
    string_message = ""
    for x in message:
        string_message += x
    ascii_string = "".join(
        chr(int(string_message[i * 8: i * 8 + 8], 2))
        for i in range(len(string_message) // 8)
    )
    return ascii_string


def compare(img1, img2):
    x = min(img1.shape[0], img2.shape[0])
    y = min(img1.shape[1], img2.shape[1])
    for i in range(x):
        for j in range(y):
            if img1[i, j] != img2[i, j]:
                print(i, j)


def compare_arr(img1, img2):
    x = min(len(img1), len(img2))
    for i in range(x):
        if img1[i] != img2[i]:
            print(i)


"""##Check Quility"""


def get_psnr(input_Path_1, input_Path_2):
    image1 = cv2.imread(input_Path_1)
    image2 = cv2.imread(input_Path_2)
    print("PSNR : ", psnr(image1, image2))
    return psnr(image1, image2)


def get_ssim(input_Path_1, input_Path_2):
    image1 = cv2.imread(input_Path_1)
    image2 = cv2.imread(input_Path_2)
    print("SSIM : ", ssim(image1, image2)[0])
    return ssim(image1, image2)[0]


def get_uqi(input_Path_1, input_Path_2):
    image1 = cv2.imread(input_Path_1)
    image2 = cv2.imread(input_Path_2)
    print("UQI : ", uqi(image1, image2))
    return uqi(image1, image2)


def check_capasity(roi, lenght):
    x, y = roi.shape
    print("capasity(in chars): ", (x*y)/8)
    max_Characters = math.floor((x*y)/8)
    print("Length of Binary : ", lenght)
    print("Hidden Ration = ", ((lenght * 100) / (x*y)))
    if(lenght > (x*y)):
        raise NotEnoughCapasity(
            "Oops!!! Failed to Embedd Message into given Image \n\n Max Capasity is of '{}' Characters".format(max_Characters))

# endregion


def encode(input_Path, text, output_Path='', output_FractionalPath='', progressBar=None):
    begin_time = datetime.datetime.now()
    image = cv2.imread(input_Path)
    binary_image = skin_detection(image)
    hole_filled_image = FillHole(binary_image)
    y_nonzero, x_nonzero = largest_connected_component_coordinates(
        hole_filled_image)
    ROI = crop(y_nonzero, x_nonzero, binary_image)
    coeffs = get_subbands(ROI)
    max_entropy_subband = get_max_entropy_subband(coeffs)
    binary = str_to_binary(text)
    # check if we can accomodate msg inside this segmant
    check_capasity(max_entropy_subband, len(binary))
    coordinates = generate_coordinates(max_entropy_subband, len(binary))
    modified_max_entropy_band = hide_in_maxband(
        coordinates, max_entropy_subband, binary)
    modified_roi = get_modified_ROI(
        coeffs, modified_max_entropy_band, max_entropy_subband)
    int_roi, fract_roi = seperate_int_fract(modified_roi)
    modified_binary_image = merge_roi(
        binary_image, int_roi, y_nonzero, x_nonzero)
    B, G, R = cv2.split(image)
    modified_r = hide_in_R(modified_binary_image, R)
    stego_image = cv2.merge([B, G, modified_r])
    # print("Seed Value is : ", len(binary))
    # cv2.imwrite("stego_image", stego_image)
    seed_value = len(binary)

    # for Duplicate Files
    if pathlib.Path(output_Path).suffix == '.png':
        # print("Suffix is .png")
        wrt1 = cv2.imwrite('drive/MyDrive/stego'+'.png', stego_image)
    else:
        wrt1 = cv2.imwrite('drive/MyDrive/stego' + '.png', stego_image)
    if pathlib.Path(output_FractionalPath).suffix == '.npy':
        # print("Suffix is .npy")
        wrt2 = np.save('drive/MyDrive/stego' + '.npy', fract_roi)
    else:
        wrt2 = np.save('drive/MyDrive/stego' + '.npy', fract_roi)

    if not wrt1:
        raise FileError("Failed to write image '{}'".format(output_Path))
    # elif not wrt2:
    #     raise FileError("Failed to write Fractional Image '{}'".format(
    #         output_FractionalPath))
    # else:
    #     # setting for loop to set value of progress bar
    #     for i in range(101):

    #         # slowing down the loop
    #         time.sleep(0.007)

    #         # setting value to progress bar
    #         progressBar.setValue(i)
    print("Total Encoding Time : ", datetime.datetime.now() - begin_time)
    return seed_value


def decode(input_Path, input_FractionalPath, seed, progressBar=None):
    begin_time = datetime.datetime.now()
    if(int(seed) % 8 != 0):
        raise SeedNotValid("Given seed '{}' is not Valid".format(
            seed))
    stego_image = cv2.imread(input_Path)
    fract_roi = np.load(input_FractionalPath)
    B, G, R = cv2.split(stego_image)
    binary_image = get_binary_from_R(R)
    hole_filled_image = FillHole(binary_image)
    y_nonzero, x_nonzero = largest_connected_component_coordinates(
        hole_filled_image)
    ROI = crop(y_nonzero, x_nonzero, binary_image)
    modified_ROI = combine_roi(ROI, fract_roi)
    coeffs = get_subbands(modified_ROI)
    max_entropy_subband = get_max_entropy_subband(coeffs)
    coordinates = generate_coordinates(max_entropy_subband, int(seed))
    text = get_message(max_entropy_subband, coordinates)
    print("Total Decoding Time : ", datetime.datetime.now() - begin_time)
    # for i in range(101):

    #     # slowing down the loop
    #     time.sleep(0.007)

    #     # setting value to progress bar
    #     progressBar.setValue(i)

    return text

# Testing Code

In [ ]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/1.jpeg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


In [33]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/2.png'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


SkinNotDetected: ignored

In [37]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/3.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


239 925 654 1716
Shape of Subnand : 343  x  531
LH :  0.1722186110175458
HL :  0.19192905376947492
HH :  0.11824127452941134
Selected Band : HL
capasity(in chars):  22766.625
Length of Binary :  1024
Hidden Ration =  0.5622265048069268
Total Encoding Time :  0:00:13.139704
seed:  1024
DECODING started...............
239 925 654 1716
Shape of Subnand : 343  x  531
LH :  0.18585957370541645
HL :  0.23290324759938083
HH :  0.1318818341108048
Selected Band : HL
Total Decoding Time :  0:00:05.907678
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.434387985724136
SSIM :  0.9973001270092242
UQI :  0.9996565588094769
----------------for 2nd String-------------
239 925 654 1716
Shape of Subnand : 343  x  531
LH :  0.1722186110175458
HL :  0.19192905376947492
HH :  0.11824127452941134
Selected Band : HL
capasity(in chars):  22766.625
Length of Binary :  2472
Hidden Ration =  1.3572499217604717


In [6]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/4.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


737 3263 0 2036
Shape of Subnand : 1263  x  1018
LH :  0.07429278516843602
HL :  0.09317367045628411
HH :  0.05143578521167153
Selected Band : HL
capasity(in chars):  160716.75
Length of Binary :  1024
Hidden Ration =  0.07964322324835464
Total Encoding Time :  0:00:53.756614
seed:  1024
DECODING started...............
737 3263 0 2036
Shape of Subnand : 1263  x  1018
LH :  0.07939570214322943
HL :  0.10301491966470858
HH :  0.05652193360773276
Selected Band : HL
Total Decoding Time :  0:00:36.483546
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  46.133211751520626
SSIM :  0.9968782587090298
UQI :  0.9999303728510154
----------------for 2nd String-------------
737 3263 0 2036
Shape of Subnand : 1263  x  1018
LH :  0.07429278516843602
HL :  0.09317367045628411
HH :  0.05143578521167153
Selected Band : HL
capasity(in chars):  160716.75
Length of Binary :  2472
Hidden Ration =  0.1922637186

In [38]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/5.png'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


10 517 132 417
Shape of Subnand : 254  x  143
LH :  0.2282480114823455
HL :  0.2521413134475191
HH :  0.16090277432328
Selected Band : HL
capasity(in chars):  4540.25
Length of Binary :  1024
Hidden Ration =  2.8192280160784096
Total Encoding Time :  0:00:01.631977
seed:  1024
DECODING started...............
10 517 132 417
Shape of Subnand : 254  x  143
LH :  0.24800759238649164
HL :  0.3764931488505688
HH :  0.18044955819630693
Selected Band : HL
Total Decoding Time :  0:00:00.894860
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low&' - Martin Luther King Jr.
PSNR :  45.44761721586326
SSIM :  0.9978526446610809
UQI :  0.9998395096131967
----------------for 2nd String-------------
10 517 132 417
Shape of Subnand : 254  x  143
LH :  0.2282480114823455
HL :  0.2521413134475191
HH :  0.16090277432328
Selected Band : HL
capasity(in chars):  4540.25
Length of Binary :  2472
Hidden Ration =  6.805792632564286
Total Encoding Time 

In [39]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/6.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


146 688 360 979
Shape of Subnand : 271  x  310
LH :  0.3363172035007862
HL :  0.397801611621449
HH :  0.2253024341207598
Selected Band : HL
capasity(in chars):  10501.25
Length of Binary :  1024
Hidden Ration =  1.2189025116057612
Total Encoding Time :  0:00:08.367254
seed:  1024
DECODING started...............
146 688 360 979
Shape of Subnand : 271  x  310
LH :  0.3661977282196645
HL :  0.48163369995517286
HH :  0.2550287521527653
Selected Band : HL
Total Decoding Time :  0:00:03.968643
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.48292511224626
SSIM :  0.9972689149178341
UQI :  0.9996048227416154
----------------for 2nd String-------------
146 688 360 979
Shape of Subnand : 271  x  310
LH :  0.3363172035007862
HL :  0.397801611621449
HH :  0.2253024341207598
Selected Band : HL
capasity(in chars):  10501.25
Length of Binary :  2472
Hidden Ration =  2.942506844423283
Total Encoding

In [40]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/7.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.155015030520448
HL :  0.10958742319239195
HH :  0.08319961469050799
Selected Band : LH
capasity(in chars):  1955.0
Length of Binary :  1024
Hidden Ration =  6.547314578005115
Total Encoding Time :  0:00:02.675541
seed:  1024
DECODING started...............
429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.3743016971584829
HL :  0.11854737271918593
HH :  0.09192430074912289
Selected Band : LH
Total Decoding Time :  0:00:01.089540
your msg:  'I have a dream that one day every vallei shall bE Exaltad, every hill and mountain shall be made low.' - Martin Lu4her Kang Jr.
PSNR :  47.2080370008667
SSIM :  0.9970209054539282
UQI :  0.999945017407014
----------------for 2nd String-------------
429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.155015030520448
HL :  0.10958742319239195
HH :  0.08319961469050799
Selected Band : LH
capasity(in chars):  1955.0
Length of Binary :  2472
Hidden Ration =  15.805626598465473
Total Encoding Tim

NotEnoughCapasity: ignored

In [41]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/7.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.155015030520448
HL :  0.10958742319239195
HH :  0.08319961469050799
Selected Band : LH
capasity(in chars):  1955.0
Length of Binary :  1024
Hidden Ration =  6.547314578005115
Total Encoding Time :  0:00:02.648986
seed:  1024
DECODING started...............
429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.3743016971584829
HL :  0.11854737271918593
HH :  0.09192430074912289
Selected Band : LH
Total Decoding Time :  0:00:01.124196
your msg:  'I have a dream that one day every vallei shall bE Exaltad, every hill and mountain shall be made low.' - Martin Lu4her Kang Jr.
PSNR :  47.2080370008667
SSIM :  0.9970209054539282
UQI :  0.999945017407014
----------------for 2nd String-------------
429 612 498 838
Shape of Subnand : 92  x  170
LH :  0.155015030520448
HL :  0.10958742319239195
HH :  0.08319961469050799
Selected Band : LH
capasity(in chars):  1955.0
Length of Binary :  2472
Hidden Ration =  15.805626598465473
Total Encoding Tim

NotEnoughCapasity: ignored

In [42]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/8.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


405 699 298 473
Shape of Subnand : 147  x  88
LH :  0.3887844935706739
HL :  0.399981706574762
HH :  0.24570093808428467
Selected Band : HL
capasity(in chars):  1617.0
Length of Binary :  1024
Hidden Ration =  7.915893630179345
Total Encoding Time :  0:00:02.058631
seed:  1024
DECODING started...............
405 699 298 473
Shape of Subnand : 147  x  88
LH :  0.4226400515287798
HL :  0.6851390085008777
HH :  0.2795112855656667
Selected Band : HL
Total Decoding Time :  0:00:00.785733
your msg:  'I have a dream that one day every valley shalh be exalted( every hiLl and mountaan shall be made low.' - Martin Luther King Jr.
PSNR :  47.71003012106625
SSIM :  0.9970384683186486
UQI :  0.9873266356157376
----------------for 2nd String-------------
405 699 298 473
Shape of Subnand : 147  x  88
LH :  0.3887844935706739
HL :  0.399981706574762
HH :  0.24570093808428467
Selected Band : HL
capasity(in chars):  1617.0
Length of Binary :  2472
Hidden Ration =  19.109461966604822
Total Encoding Time 

NotEnoughCapasity: ignored

In [43]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/9.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


145 2606 2111 4229
Shape of Subnand : 1231  x  1059
LH :  0.05043241984156207
HL :  0.058821949346252525
HH :  0.03852028375324486
Selected Band : HL
capasity(in chars):  162953.625
Length of Binary :  1024
Hidden Ration =  0.07854995554716872
Total Encoding Time :  0:01:43.623794
seed:  1024
DECODING started...............
145 2606 2111 4229
Shape of Subnand : 1231  x  1059
LH :  0.05408679631758375
HL :  0.06720020341124347
HH :  0.04216810989622303
Selected Band : HL
Total Decoding Time :  0:00:52.090754
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.4595016410162
SSIM :  0.995010022618951
UQI :  0.9999479957290801
----------------for 2nd String-------------
145 2606 2111 4229
Shape of Subnand : 1231  x  1059
LH :  0.05043241984156207
HL :  0.058821949346252525
HH :  0.03852028375324486
Selected Band : HL
capasity(in chars):  162953.625
Length of Binary :  2472
Hidden Ration =  0.

In [44]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/10.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


293 442 484 628
Shape of Subnand : 75  x  72
LH :  0.39295133859710957
HL :  0.3988444432244064
HH :  0.2570739809923381
Selected Band : HL
capasity(in chars):  675.0
Length of Binary :  1024
Hidden Ration =  18.962962962962962
Total Encoding Time :  0:00:03.794499
seed:  1024
DECODING started...............
293 442 484 628
Shape of Subnand : 75  x  72
LH :  0.4282236239519164
HL :  0.9309447441839024
HH :  0.2918136093191715
Selected Band : HL
Total Decoding Time :  0:00:01.347862
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.39897412097523
SSIM :  0.9985817281432396
UQI :  0.9996991496686435
----------------for 2nd String-------------
293 442 484 628
Shape of Subnand : 75  x  72
LH :  0.39295133859710957
HL :  0.3988444432244064
HH :  0.2570739809923381
Selected Band : HL
capasity(in chars):  675.0
Length of Binary :  2472
Hidden Ration =  45.77777777777778
Total Encoding Time :  

NotEnoughCapasity: ignored

In [45]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/11.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


0 196 0 479
Shape of Subnand : 98  x  240
LH :  0.14081465817999847
HL :  0.07002356296880577
HH :  0.04610011420058618
Selected Band : LH
capasity(in chars):  2940.0
Length of Binary :  1024
Hidden Ration =  4.35374149659864
Total Encoding Time :  0:00:01.203822
seed:  1024
DECODING started...............
0 196 0 479
Shape of Subnand : 98  x  240
LH :  0.29602829903535477
HL :  0.07516505493722574
HH :  0.05163493423289976
Selected Band : LH
Total Decoding Time :  0:00:00.662564
your msg:  'I have a dream That one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.515653815716774
SSIM :  0.9962386439883377
UQI :  0.9998219904591826
----------------for 2nd String-------------
0 196 0 479
Shape of Subnand : 98  x  240
LH :  0.14081465817999847
HL :  0.07002356296880577
HH :  0.04610011420058618
Selected Band : LH
capasity(in chars):  2940.0
Length of Binary :  2472
Hidden Ration =  10.510204081632653
Total Encoding Time :  

NotEnoughCapasity: ignored

In [46]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/12.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


234 639 0 959
Shape of Subnand : 203  x  480
LH :  0.3295300764266041
HL :  0.2772241368574104
HH :  0.1660731900056762
Selected Band : LH
capasity(in chars):  12180.0
Length of Binary :  1024
Hidden Ration =  1.0509031198686372
Total Encoding Time :  0:00:04.300547
seed:  1024
DECODING started...............
234 639 0 959
Shape of Subnand : 203  x  480
LH :  0.39644880728036186
HL :  0.29763756990741846
HH :  0.18713743405708072
Selected Band : LH
Total Decoding Time :  0:00:02.418840
your msg:  'I have a dream that one day every vallEy shall be exalted, eveby hill and mOqntain shall be made low.' - Martin Luther King Jr.
PSNR :  47.26878540102945
SSIM :  0.9959575156141008
UQI :  0.9998896992550245
----------------for 2nd String-------------
234 639 0 959
Shape of Subnand : 203  x  480
LH :  0.3295300764266041
HL :  0.2772241368574104
HH :  0.1660731900056762
Selected Band : LH
capasity(in chars):  12180.0
Length of Binary :  2472
Hidden Ration =  2.5369458128078817
Total Encoding Ti

In [47]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/13.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


84 540 67 448
Shape of Subnand : 228  x  191
LH :  0.11990073020959195
HL :  0.12035701449302888
HH :  0.0727372803131765
Selected Band : HL
capasity(in chars):  5443.5
Length of Binary :  1024
Hidden Ration =  2.3514283089923764
Total Encoding Time :  0:00:01.878026
seed:  1024
DECODING started...............
84 540 67 448
Shape of Subnand : 228  x  191
LH :  0.1275933070976634
HL :  0.21891346030475403
HH :  0.08039859811367141
Selected Band : HL
Total Decoding Time :  0:00:01.048506
your msg:  'I have a dream that one day every valley shall be ex`lted, every hill and mouNtain shalL be made low.' - MartiN Luther King Jr.
PSNR :  45.700772358924596
SSIM :  0.9984276246776153
UQI :  0.9998755072224824
----------------for 2nd String-------------
84 540 67 448
Shape of Subnand : 228  x  191
LH :  0.11990073020959195
HL :  0.12035701449302888
HH :  0.0727372803131765
Selected Band : HL
capasity(in chars):  5443.5
Length of Binary :  2472
Hidden Ration =  5.676494902176908
Total Encoding T

In [49]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/14.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


0 824 0 799
Shape of Subnand : 412  x  400
LH :  0.15548362718333583
HL :  0.1850335742164096
HH :  0.10186946560833575
Selected Band : HL
capasity(in chars):  20600.0
Length of Binary :  1024
Hidden Ration =  0.6213592233009708
Total Encoding Time :  0:00:05.159785
seed:  1024
DECODING started...............
0 824 0 799
Shape of Subnand : 412  x  400
LH :  0.16692012113850133
HL :  0.22567105672895973
HH :  0.113314108137855
Selected Band : HL
Total Decoding Time :  0:00:03.358496
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.43066015681425
SSIM :  0.9960130530574735
UQI :  0.9999562998849644
----------------for 2nd String-------------
0 824 0 799
Shape of Subnand : 412  x  400
LH :  0.15548362718333583
HL :  0.1850335742164096
HH :  0.10186946560833575
Selected Band : HL
capasity(in chars):  20600.0
Length of Binary :  2472
Hidden Ration =  1.5
Total Encoding Time :  0:00:05.14135

In [50]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/15.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


1164 1269 672 822
Shape of Subnand : 53  x  75
LH :  0.5485642812449231
HL :  0.44895810016277216
HH :  0.30309103293001205
Selected Band : LH
capasity(in chars):  496.875
Length of Binary :  1024
Hidden Ration =  25.761006289308177
Total Encoding Time :  0:00:11.840518
seed:  1024
DECODING started...............
1164 1269 672 822
Shape of Subnand : 53  x  75
LH :  1.210358373241255
HL :  0.49176640694504975
HH :  0.38559929533000703
Selected Band : LH
Total Decoding Time :  0:00:02.408795
your msg:  'I have a dream that one day every vaLley shall be exalted, every hill and mounTain sHAll be made los.' - Martin Luther King Jr.
PSNR :  47.43701592991684
SSIM :  0.9978632986708127
UQI :  0.9997601278986367
----------------for 2nd String-------------
1164 1269 672 822
Shape of Subnand : 53  x  75
LH :  0.5485642812449231
HL :  0.44895810016277216
HH :  0.30309103293001205
Selected Band : LH
capasity(in chars):  496.875
Length of Binary :  2472
Hidden Ration =  62.18867924528302
Total Enco

NotEnoughCapasity: ignored

In [51]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/16.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


LargestComponentNotFound: ignored

In [53]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/17.jpeg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


SkinNotDetected: ignored

In [54]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/18.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


138 899 0 1199
Shape of Subnand : 381  x  600
LH :  0.26816158962660813
HL :  0.271669387456415
HH :  0.15036007715010546
Selected Band : HL
capasity(in chars):  28575.0
Length of Binary :  1024
Hidden Ration =  0.4479440069991251
Total Encoding Time :  0:00:09.746321
seed:  1024
DECODING started...............
138 899 0 1199
Shape of Subnand : 381  x  600
LH :  0.2870760354190753
HL :  0.31482087741719444
HH :  0.16914793460074476
Selected Band : HL
Total Decoding Time :  0:00:05.926804
your msg:  'I have a dream that on% day every valley shall be exalted, every hill and mountain s`all be made low.& - Martin Luthdr King Jr.
PSNR :  47.24276438543197
SSIM :  0.998905753130995
UQI :  0.9996898701946136
----------------for 2nd String-------------
138 899 0 1199
Shape of Subnand : 381  x  600
LH :  0.26816158962660813
HL :  0.271669387456415
HH :  0.15036007715010546
Selected Band : HL
capasity(in chars):  28575.0
Length of Binary :  2472
Hidden Ration =  1.0813648293963254
Total Encoding

In [55]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/19.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


0 1265 0 1599
Shape of Subnand : 633  x  800
LH :  0.3381343454161874
HL :  0.28602762373096874
HH :  0.19616546646003558
Selected Band : LH
capasity(in chars):  63300.0
Length of Binary :  1024
Hidden Ration =  0.20221169036334913
Total Encoding Time :  0:00:40.963822
seed:  1024
DECODING started...............
0 1265 0 1599
Shape of Subnand : 633  x  800
LH :  0.37398762935207014
HL :  0.31076175831270475
HH :  0.22122982168268177
Selected Band : LH
Total Decoding Time :  0:00:31.263126
your msg:  'I have a dream that one day eveRy valley shall be exalted, everY hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.455746955557814
SSIM :  0.9985634749056086
UQI :  0.9999681329058286
----------------for 2nd String-------------
0 1265 0 1599
Shape of Subnand : 633  x  800
LH :  0.3381343454161874
HL :  0.28602762373096874
HH :  0.19616546646003558
Selected Band : LH
capasity(in chars):  63300.0
Length of Binary :  2472
Hidden Ration =  0.4881516587677725
Total Encod

In [56]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/20.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


240 338 0 507
Shape of Subnand : 49  x  254
LH :  0.3618372454562345
HL :  0.2705443015427945
HH :  0.19105425809394647
Selected Band : LH
capasity(in chars):  1555.75
Length of Binary :  1024
Hidden Ration =  8.227542985698216
Total Encoding Time :  0:00:00.977284
seed:  1024
DECODING started...............
240 338 0 507
Shape of Subnand : 49  x  254
LH :  0.6380612761604043
HL :  0.29507310121527097
HH :  0.22337668414451645
Selected Band : LH
Total Decoding Time :  0:00:00.469449
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.336205387015184
SSIM :  0.9978306329454116
UQI :  0.9999435297755578
----------------for 2nd String-------------
240 338 0 507
Shape of Subnand : 49  x  254
LH :  0.3618372454562345
HL :  0.2705443015427945
HH :  0.19105425809394647
Selected Band : LH
capasity(in chars):  1555.75
Length of Binary :  2472
Hidden Ration =  19.8618029889121
Total Encoding Time :

NotEnoughCapasity: ignored

In [57]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/21.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


809 1114 201 432
Shape of Subnand : 153  x  116
LH :  0.3572913867173559
HL :  0.40626837356171874
HH :  0.25381846286275656
Selected Band : HL
capasity(in chars):  2218.5
Length of Binary :  1024
Hidden Ration =  5.769664187514086
Total Encoding Time :  0:00:15.909277
seed:  1024
DECODING started...............
809 1114 201 432
Shape of Subnand : 153  x  116
LH :  0.39217787498997664
HL :  0.6346827701265908
HH :  0.28869013364614393
Selected Band : HL
Total Decoding Time :  0:00:06.673693
your msg:  'I have a dream that one day every valley shall be eXalted, every `ill and mountaij shall be made low.' - Martin Luther King Jr.
PSNR :  47.34208828252473
SSIM :  0.9978480840702852
UQI :  0.9997194028733261
----------------for 2nd String-------------
809 1114 201 432
Shape of Subnand : 153  x  116
LH :  0.3572913867173559
HL :  0.40626837356171874
HH :  0.25381846286275656
Selected Band : HL
capasity(in chars):  2218.5
Length of Binary :  2472
Hidden Ration =  13.928329952670724
Total En

NotEnoughCapasity: ignored

In [14]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/22.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


0 548 0 914
Shape of Subnand : 274  x  457
LH :  0.10527732214471644
HL :  0.12338395776029462
HH :  0.053221959210841184
Selected Band : HL
capasity(in chars):  15652.25
Length of Binary :  1024
Hidden Ration =  0.8177738024884601
Total Encoding Time :  0:00:04.534011
seed:  1024
DECODING started...............
0 548 0 914
Shape of Subnand : 274  x  457
LH :  0.11058705903406939
HL :  0.16529003963496972
HH :  0.05833870943309325
Selected Band : HL
Total Decoding Time :  0:00:02.567133
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.30700465089565
SSIM :  0.9955412265247808
UQI :  0.9999460377141308
----------------for 2nd String-------------
0 548 0 914
Shape of Subnand : 274  x  457
LH :  0.10527732214471644
HL :  0.12338395776029462
HH :  0.053221959210841184
Selected Band : HL
capasity(in chars):  15652.25
Length of Binary :  2472
Hidden Ration =  1.9741570700697983
Total Encodin

In [15]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/23.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


378 556 867 958
Shape of Subnand : 89  x  46
LH :  0.7254707354234359
HL :  0.7062247375193356
HH :  0.4432872909024964
Selected Band : LH
capasity(in chars):  511.75
Length of Binary :  1024
Hidden Ration =  25.01221299462628
Total Encoding Time :  0:00:04.820455
seed:  1024
DECODING started...............
378 556 867 958
Shape of Subnand : 89  x  46
LH :  1.4018143843603288
HL :  0.7775541158267674
HH :  0.5748018195401052
Selected Band : LH
Total Decoding Time :  0:00:02.489365
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.352849454081976
SSIM :  0.9996989403680475
UQI :  0.9998470175355566
----------------for 2nd String-------------
378 556 867 958
Shape of Subnand : 89  x  46
LH :  0.7254707354234359
HL :  0.7062247375193356
HH :  0.4432872909024964
Selected Band : LH
capasity(in chars):  511.75
Length of Binary :  2472
Hidden Ration =  60.38104543234001
Total Encoding Time :  

NotEnoughCapasity: ignored

In [16]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/24.jpeg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


365 583 28 163
Shape of Subnand : 109  x  68
LH :  0.3750844570452723
HL :  0.46660387325676567
HH :  0.24485639600219533
Selected Band : HL
capasity(in chars):  926.5
Length of Binary :  1024
Hidden Ration =  13.815434430652996
Total Encoding Time :  0:00:04.164551
seed:  1024
DECODING started...............
365 583 28 163
Shape of Subnand : 109  x  68
LH :  0.4083507755709521
HL :  0.8863352042866722
HH :  0.2782739671972869
Selected Band : HL
Total Decoding Time :  0:00:01.632640
your msg:  'I have a dream that one `!y every valley shall be exalted, every (Ill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.30388859175301
SSIM :  0.9988044156592046
UQI :  0.9983630632437196
----------------for 2nd String-------------
365 583 28 163
Shape of Subnand : 109  x  68
LH :  0.3750844570452723
HL :  0.46660387325676567
HH :  0.24485639600219533
Selected Band : HL
capasity(in chars):  926.5
Length of Binary :  2472
Hidden Ration =  33.351322180248246
Total Encoding Time 

NotEnoughCapasity: ignored

In [26]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/25.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


451 775 601 1179
Shape of Subnand : 162  x  289
LH :  0.260377102906702
HL :  0.2667018782155522
HH :  0.16980433766444267
Selected Band : HL
capasity(in chars):  5852.25
Length of Binary :  1024
Hidden Ration =  2.18719295997266
Total Encoding Time :  0:00:11.597993
seed:  1024
DECODING started...............
("local variable 'value' referenced before assignment",)
your msg:  Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made mahor contributions to mathematics, cryppanalysis, logic, philosophy, and mathematical biology and also to tHe new areas later named computer science, ckgnitive science, artificial intelligence, and artificia, life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. Af4er graduating in 1934, he was elected to a fellowship at King's College (his college since 1931

In [18]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/26.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


109 249 859 1033
Shape of Subnand : 70  x  87
LH :  0.5373100522199179
HL :  0.5232199074089537
HH :  0.3462158290757636
Selected Band : LH
capasity(in chars):  761.25
Length of Binary :  1024
Hidden Ration =  16.814449917898195
Total Encoding Time :  0:00:06.084922
seed:  1024
DECODING started...............
109 249 859 1033
Shape of Subnand : 70  x  87
LH :  1.0683418447637578
HL :  0.6146724678064219
HH :  0.45145342835583413
Selected Band : LH
Total Decoding Time :  0:00:02.617235
your msg:  'I have a dream that one day every valley shall be exalted, evgry hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.55121528993217
SSIM :  0.9986724015299672
UQI :  0.9946186550621214
----------------for 2nd String-------------
109 249 859 1033
Shape of Subnand : 70  x  87
LH :  0.5373100522199179
HL :  0.5232199074089537
HH :  0.3462158290757636
Selected Band : LH
capasity(in chars):  761.25
Length of Binary :  2472
Hidden Ration =  40.59113300492611
Total Encoding Time

NotEnoughCapasity: ignored

In [19]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/27.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


LargestComponentNotFound: ignored

In [20]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/28.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


LargestComponentNotFound: ignored

In [27]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/29.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


109 666 0 1199
Shape of Subnand : 279  x  600
LH :  0.3599577187447395
HL :  0.28220121955720556
HH :  0.1696378702128421
Selected Band : LH
capasity(in chars):  20925.0
Length of Binary :  1024
Hidden Ration =  0.6117084826762246
Total Encoding Time :  0:00:05.767252
seed:  1024
DECODING started...............
109 666 0 1199
Shape of Subnand : 279  x  600
LH :  0.410755535849806
HL :  0.30363925186416835
HH :  0.19164098593934947
Selected Band : LH
Total Decoding Time :  0:00:03.710825
your msg:  'I have a dream that one day every valley shall be exalted, every hill and moundain shall be made low.' - Martin Luther King Jr.
PSNR :  47.704167366284764
SSIM :  0.9983644355702848
UQI :  0.9995361250332607
----------------for 2nd String-------------
109 666 0 1199
Shape of Subnand : 279  x  600
LH :  0.3599577187447395
HL :  0.28220121955720556
HH :  0.1696378702128421
Selected Band : LH
capasity(in chars):  20925.0
Length of Binary :  2472
Hidden Ration =  1.4767025089605734
Total Encodin

In [28]:
if __name__ == "__main__":

    # # Encode
    # ip_file = str(
    #     input('\nEnter cover image name(path)(with extension): '))
    ip_file = 'drive/MyDrive/30.jpg'
    # text = str(input('Enter secret data: '))
    text1 = "'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr."
    text2 = "This file is created to hide text. Tomorrow is Republic day for india. How ARE 12345 you. password1: ` 1 2 3 4 5 6 7 8 9 0 - = . password2: ~ ! @ # $ % ^ & * ( ) _ + . password3: ABCDEFGHIJKLMNOPQRSTUVWXYZ. password4: abcdefghijklmnopqrstuvwxyz. password5: aVrt@56Hm Please change the size of the cover image."
    text3 = "Alan Turing, in full Alan Mathison Turing, (born June 23, 1912, London, England-died June 7, 1954, Wilmslow, Cheshire), British mathematician and logician who made major contributions to mathematics, cryptanalysis, logic, philosophy, and mathematical biology and also to the new areas later named computer science, cognitive science, artificial intelligence, and artificial life. The son of a civil servant, Turing was educated at a top private school. He entered the University of Cambridge to study mathematics in 1931. After graduating in 1934, he was elected to a fellowship at King's College (his college since 1931) in recognition of his research in probability theory. In 1936 Turing'jss seminal paper On Computable Numbers, with an Application to the Entscheidungsproblem [Decision Problem]” was recommended for publication by the American mathematical logician Alonzo Church, who had himself just published a paper that reached the same conclusion as Turing'jss, although by a different method. Turing's method (but not so much Church's) had profound significance for the emerging science of computing. Later that year Turing moved to Princeton University to study for a Ph.D. in mathematical logic under Church'jss direction (completed in 1938). What mathematicians called an 'effective' method for solving a problem was simply one that could be carried by a human mathematical clerk working by rote. In Turing'jss time, those rote-workers were in fact called “computers,” and human computers carried out some aspects of the work later done by electronic computers. The Entscheidungsproblem sought an effective method for solving the fundamental mathematical problem of determining exactly which mathematical statements are provable within a given formal mathematical system and which are not. A method for determining this is called a decision method. In 1936 Turing and Church independently showed that, in general, the Entscheidungsproblem problem has no resolution, proving that no consistent formal system of arithmetic has an effective decision method. In fact, Turing and Church showed that even some purely logical systems, considerably weaker than arithmetic, have no effective decision method. This result and others-notably mathematician-logician Kurt Godel's incompleteness results-dashed the hopes, held by some mathematicians, of discovering a formal system that would reduce the whole of mathematics to methods that (human) computers could carry out. It was in the course of his work on the Entscheidungsproblem that Turing invented the universal Turing machine, an abstract computing machine that encapsulates the fundamental logical principles of the digital computer. An important step in Turing's argument about the Entscheidungsproblem was the claim, now called the Church-Turing thesis, that everything humanly computable can also be computed by the universal Turing machine. The claim is important because it marks out the limits of human computation. Church in his work used instead the thesis that all human-computable functions are identical to what he called lambda-definable functions (functions on the positive integers whose values can be calculated by a process of repeated substitution). Turing showed in 1936 that Church's thesis was equivalent to his own, by proving that every lambda-definable function is computable by the universal Turing machine and vice versa. In a review of Turing's work, Church acknowledged the superiority of Turing's formulation of the thesis over his own (which made no reference to computing machinery), saying that the concept of computability by a Turing machine 'has the advantage of making the identification with effectiveness jsevident immediately.'"

    #text3 = "Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Aenean commodo ligula eget dolor. Aenean massa. Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Donec quam felis, ultricies nec, pellentesque eu, pretium quis, sem. Nulla consequat massa quis enim. Donec pede justo, fringilla vel, aliquet nec, vulputate eget, arcu. In enim justo, rhoncus ut, imperdiet a"

    op_file = 'stego.png'
    seed1 = encode(ip_file, text1, op_file, op_file)
    print("seed: ", seed1)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed1
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 2nd String-------------")
    seed2 = encode(ip_file, text2, op_file, op_file)
    print("seed: ", seed2)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    pwd = seed2
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)

    print("----------------for 3rd String-------------")
    seed3 = encode(ip_file, text3, op_file, op_file)
    print("seed: ", seed3)
    print("DECODING started...............")
    ip_file1 = 'drive/MyDrive/stego.png'
    ip_file2 = 'drive/MyDrive/stego.npy'
    #pwd = 30080
    pwd = seed3
    try:
        data = decode(ip_file1, ip_file2, pwd)
    except Exception as e:
        print(e.args)
    print("your msg: ", data)
    get_psnr(ip_file, ip_file1)
    get_ssim(ip_file, ip_file1)
    get_uqi(ip_file, ip_file1)


805 2047 1158 2687
Shape of Subnand : 621  x  765
LH :  0.14484994663256295
HL :  0.14838842989319237
HH :  0.09064491390440153
Selected Band : HL
capasity(in chars):  59383.125
Length of Binary :  1024
Hidden Ration =  0.2155494511277404
Total Encoding Time :  0:00:48.404720
seed:  1024
DECODING started...............
805 2047 1158 2687
Shape of Subnand : 621  x  765
LH :  0.15478204801616022
HL :  0.16963084437681764
HH :  0.1004683480500722
Selected Band : HL
Total Decoding Time :  0:00:25.538110
your msg:  'I have a dream that one day every valley shall be exalted, every hill and mountain shall be made low.' - Martin Luther King Jr.
PSNR :  47.46491645328725
SSIM :  0.9956603470531175
UQI :  0.9985944936854412
----------------for 2nd String-------------
805 2047 1158 2687
Shape of Subnand : 621  x  765
LH :  0.14484994663256295
HL :  0.14838842989319237
HH :  0.09064491390440153
Selected Band : HL
capasity(in chars):  59383.125
Length of Binary :  2472
Hidden Ration =  0.5203498468